In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os
import argparse
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import auc,brier_score_loss
import numpy as np

from Utils.treino import *
from Utils.init_redes import *
from Utils.redes import *
from Utils.data_set import *
from Utils.graficos import *
from Utils.loading import *

cuda


In [2]:
def testa_acuracia(model, testloader, T):
    # Calcula a acuracia da rede
    model.eval()
    correct = 0
    correct2 = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            images, labels = data
            # calculate outputs by running images through the network
            outputs = model(images.to("cuda"))
            outputs = outputs.to("cpu")
            outputs_numpy = outputs.to("cpu").numpy()

            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            _, predicted2 = torch.max((outputs.data)/T, 1)

            total += labels.size(0)

            correct += (predicted == labels).sum().item()
            correct2 += (predicted2 == labels).sum().item()

    acuracia = round(100 * correct / total,3)
    acuracia2 = round(100 * correct2 / total,3)
    return acuracia, acuracia2

def caculate_output(nets, images):
    models = list()
    for model in nets:    
        models.append(model(images))
    return models

def calculate_correct(predicted, labels):
    correct_aux = (predicted == labels.to("cuda")).sum().item()
    return round(1-correct_aux/10000,4)

def calculate_predicted(mean_list):
    uncs_max, predicted_aux = torch.max(mean_list.data, 1)
    uncs_var = torch.var(mean_list,1)
    uncs_entr = torch.special.entr(mean_list)
    uncs_sum_entr = -torch.sum(uncs_entr, dim=-1)
    return uncs_max, uncs_var, uncs_sum_entr, predicted_aux

def correct_class(y_pred,y_true):
    '''Returns a bool tensor indicating if each prediction is correct'''
    with torch.no_grad():
        #y_pred = torch.argmax(y_pred, -1) # não entendi para que serve
        correct = (y_pred==y_true)
    return correct

def correct_total(y_pred,y_true):
    '''Returns the number of correct predictions in a batch'''
    with torch.no_grad():
        correct = correct_class(y_pred,y_true)
        correct_total = torch.sum(correct).item()
    return correct_total


def get_n_biggest(vec,n):
    '''Returns the indexes of the N biggest values in vec'''
    if 0<n<1:
        n = int(n*vec.size(0))
    unc = torch.argsort(vec, descending = True)
    return unc[0:n]

def masked_coverage(y_pred,y_true, uncertainty, coverage):
    
    #dk_mask = unc_utils.dontknow_mask(uncertainty, coverage)
    #y_pred, y_true = torch.masked_select(y_pred,1-dk_mask),torch.masked_select(y_true,1-dk_mask)#apply_mask(y_pred,y_true,1-dk_mask)
    N = round((coverage)*uncertainty.shape[0])
    id = get_n_biggest(uncertainty,N)
    y_pred = y_pred[id]
    y_true = y_true[id]
    
    return y_pred,y_true

def acc_coverage(y_pred,y_true, uncertainty, coverage):
    '''Returns the total accuracy of model in some dataset excluding the 1-c most uncertain samples'''
    y_pred,y_true = masked_coverage(y_pred,y_true, uncertainty, coverage)
    acc = correct_total(y_pred,y_true)/y_true.size(0)
    return acc

def error_coverage(y_pred,y_true, uncertainty, coverage):
    '''Returns the 0-1 loss of model in some dataset excluding the 1-c most uncertain samples'''
    return 1-acc_coverage(y_pred,y_true, uncertainty, coverage)

def RC_curve(y_pred,y_true,uncertainty, risk = error_coverage, c_list = np.arange(0.05,1.05,0.05)):
    ''' Returns an array with the accuracy of the model in the data dataset
     excluding the most uncertain (total number set by the coverage) samples.
     Each item in the output array is the accuracy when the coverage is given by same item in c_list'''

    risk_list = np.array([])
    with torch.no_grad():
        for c in c_list:
            r = risk(y_pred,y_true, uncertainty, c)
            risk_list = np.append(risk_list,r)
    return risk_list

def plot_graph(X, Y, title = "", xlabel = "", ylabel = "", line = True):
    fig1 = plt.figure()
    fig1.set_size_inches(25.5, 15.5)
    ax1 = fig1.add_subplot()
    fig1.subplots_adjust(top=0.85)
    ax1.set_title(title)
    ax1.set_xlabel(xlabel)
    ax1.set_ylabel(ylabel)
    if line == True:
        ax1.plot(X,Y)
    else:
        ax1.scatter(X,Y)
    pos1 = ax1.get_position()
    ax1.set_position([pos1.x0, pos1.y0, pos1.width * 0.9, pos1.height])
    ax1.legend(loc='center right', bbox_to_anchor=(1.25, 0.5))
    plt.grid()
    #fig1.set_size_inches(8.5, 5.5)
    return fig1

# Plota mais de um grafico para o mesmo X
def plot_mutiple_graph(X, Y, n_graphs, title = "", xlabel = "", ylabel = "", line = True, label = "{}", multi_label=False):    
    if multi_label == True:
        label = label.replace(" ", "")
        label = label.split(",")
    fig1 = plt.figure()
    fig1.set_size_inches(25.5, 15.5)
    ax1 = fig1.add_subplot()
    fig1.subplots_adjust(top=0.85)
    ax1.set_title(title)
    ax1.set_xlabel(xlabel)
    ax1.set_ylabel(ylabel)
    for i in range(n_graphs):
        if line == True:
            if multi_label == True:
                ax1.plot(X, Y[i], label=label[i])
            else:
                ax1.plot(X, Y[i], label=label.format(i+1))
        else:
            if multi_label == True:
                ax1.scatter(X, Y[i], label=label[i])
            else:
                ax1.scatter(X, Y[i], label=label.format(i+1))
    pos1 = ax1.get_position()
    ax1.set_position([pos1.x0, pos1.y0, pos1.width * 0.9, pos1.height])
    ax1.legend(loc='center right', bbox_to_anchor=(1.25, 0.5))
    plt.grid()
    #fig1.set_size_inches(8.5, 5.5)
    return fig1

In [3]:
data_set = "Cifar100"

if data_set == "Cifar10":
        transform_train, transform_test, batch_size, trainset, trainloader, testset, testloader, classes = cifar10()
        rede = "Resnet_hydra"
elif data_set == "Cifar100":
    transform_train, transform_test, batch_size, trainset, trainloader, testset, testloader= cifar100()
    rede = "Resnet100_hydra"

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# RENSET

models = load_cifar10(1)

correct_list = list()
covarege_list = list()
auc_list = list()

n_redes = 1

outputs = torch.tensor([]).to("cuda")
labels_cat = torch.tensor([])

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        images, labels = data

        outputs_0 = caculate_output(models[0:n_redes],images.to("cuda"))
        outputs_0 = torch.stack(tuple(outputs_0),dim=0)
        outputs = torch.cat((outputs,outputs_0),dim=1)
        labels_cat = torch.cat((labels_cat,labels),dim=0) # Junta as respostas

valores = np.arange(0.01,5.01,0.01)

for valor in valores:
    T = valor

    output0 = outputs[0]/T

    outputs_softmax = torch.nn.functional.softmax(output0, dim=-1)

    uncs_max, _, _, predict = calculate_predicted(outputs_softmax)

    correct = calculate_correct(predict, labels_cat)
    covarege = RC_curve(predict,labels_cat.to("cuda"),uncs_max)
    correct_list.append(correct)
    covarege_list.append(covarege)

    area_sobre_curva = auc(np.arange(0.05,1.05,0.05), covarege)
    auc_list.append(area_sobre_curva)
    print(round((1-correct)*100,2), " ",area_sobre_curva, "Valor de T: ", T)

0.93   0.9418072688822187 Valor de T:  0.01
0.93   0.9416076643016426 Valor de T:  0.02
0.93   0.9414968006799701 Valor de T:  0.03
0.93   0.9415586337080531 Valor de T:  0.04
0.93   0.9418450975628258 Valor de T:  0.05
0.93   0.9417522907862691 Valor de T:  0.060000000000000005
0.93   0.9418363669878452 Valor de T:  0.06999999999999999
0.93   0.9419825274940058 Valor de T:  0.08
0.93   0.941813622787601 Valor de T:  0.09
0.93   0.9422445859910643 Valor de T:  0.09999999999999999
0.93   0.9420881440996224 Valor de T:  0.11
0.93   0.9421159651674434 Valor de T:  0.12
0.93   0.9422176624978907 Valor de T:  0.13
0.93   0.9420885200716894 Valor de T:  0.14
0.93   0.9418608216589911 Valor de T:  0.15000000000000002
0.93   0.9417833216589911 Valor de T:  0.16
0.93   0.9423016153097847 Valor de T:  0.17
0.93   0.9424453454685148 Valor de T:  0.18000000000000002
0.93   0.9423237184843879 Valor de T:  0.19
0.93   0.942503321658991 Valor de T:  0.2
0.93   0.9424734407066101 Valor de T:  0.210000

In [7]:
import pickle 

with open("./Data_sets/pickle/correct_list_resnet.pickle", "wb") as f:
    pickle.dump(correct_list, f)
    
with open("./Data_sets/pickle/covarege_list_resnet.pickle", "wb") as f:
    pickle.dump(covarege_list, f)

with open("./Data_sets/pickle/auc_list_resnet.pickle", "wb") as f:
    pickle.dump(auc_list, f)

In [7]:
# ENEMBLE T IGUAIS

models = load_cifar100(2)

correct_list = list()
covarege_list1 = list()
auc_list = list()

n_redes = 2

outputs = torch.tensor([]).to("cuda")
labels_cat = torch.tensor([])

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        images, labels = data

        outputs_0 = caculate_output(models[0:n_redes],images.to("cuda"))
        outputs_0 = torch.stack(tuple(outputs_0),dim=0)
        outputs = torch.cat((outputs,outputs_0),dim=1)
        labels_cat = torch.cat((labels_cat,labels),dim=0) # Junta as respostas

valores = np.arange(0.01,5.01,0.01)

for valor in valores:
    T = valor

    output0 = outputs[0]/T
    output1 = outputs[1]/T
    outputs_stack = torch.stack((output0,output1),dim=0)
    
    
    #outputs_softmax = torch.nn.functional.softmax(outputs_stack, dim=-1)
    #outputs_mean = torch.mean(outputs_softmax, dim=0)

    #uncs_max, _, _, predict = calculate_predicted(outputs_mean)

    outputs_mean = torch.mean(outputs_stack, dim=0)
    outputs_softmax = torch.nn.functional.softmax(outputs_mean, dim=-1)

    uncs_max, _, _, predict = calculate_predicted(outputs_softmax)

    correct = calculate_correct(predict, labels_cat)
    covarege = RC_curve(predict,labels_cat.to("cuda"),uncs_max)
    correct_list.append(correct)
    covarege_list1.append(covarege)

    area_sobre_curva = auc(np.arange(0.05,1.05,0.05), covarege)
    auc_list.append(area_sobre_curva)
    print(round((1-correct)*100,2), " ",area_sobre_curva, "Valor de T: ", T)

78.03   0.19867095364817497 Valor de T:  0.01
78.03   0.18555844127750476 Valor de T:  0.02
78.03   0.1728158623613229 Valor de T:  0.03
78.03   0.15988498468507759 Valor de T:  0.04
78.03   0.14991254930999512 Valor de T:  0.05
78.03   0.14332569051519517 Valor de T:  0.060000000000000005
78.03   0.13457489398114786 Valor de T:  0.06999999999999999
78.03   0.12543370438857665 Valor de T:  0.08
78.03   0.11838596777878124 Valor de T:  0.09
78.03   0.11121539071695419 Valor de T:  0.09999999999999999
78.03   0.10333374264280609 Valor de T:  0.11
78.03   0.09839072552103897 Valor de T:  0.12
78.03   0.094621897543461 Valor de T:  0.13
78.03   0.09181565300583487 Valor de T:  0.14
78.03   0.0890613964862107 Valor de T:  0.15000000000000002
78.03   0.08494248350162048 Valor de T:  0.16
78.03   0.0817222424561163 Valor de T:  0.17
78.03   0.07961557325543395 Valor de T:  0.18000000000000002
78.03   0.07723462739573807 Valor de T:  0.19
78.03   0.07521334354070422 Valor de T:  0.2
78.03   0.

In [5]:
import pickle 

with open("./Data_sets/pickle/Cifar100/Meansoft/correct_list_ensemble.pickle", "wb") as f:
    pickle.dump(correct_list, f)
    
with open("./Data_sets/pickle/Cifar100/Meansoft/covarege_list_ensemble.pickle", "wb") as f:
    pickle.dump(covarege_list1, f)

with open("./Data_sets/pickle/Cifar100/Meansoft/auc_list_ensemble.pickle", "wb") as f:
    pickle.dump(auc_list, f)

In [6]:
# ENEMBLE T DIFERENTES

models = load_cifar100(2)

correct_list_Tdif = list()
covarege_list1_Tdif = list()
auc_list_Tdif = list()

n_redes = 2

outputs = torch.tensor([]).to("cuda")
labels_cat = torch.tensor([])

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        images, labels = data

        outputs_0 = caculate_output(models[0:n_redes],images.to("cuda"))
        outputs_0 = torch.stack(tuple(outputs_0),dim=0)
        outputs = torch.cat((outputs,outputs_0),dim=1)
        labels_cat = torch.cat((labels_cat,labels),dim=0) # Junta as respostas

valores1 = np.arange(0.01,5.01,0.01)
valores2 = np.arange(0.01,5.01,0.01)
i = 0

for valor1 in valores1:
    T1 = valor1
    print(i, end=" ")
    i += 1
    for valor2 in valores2:
        T2 = valor2

        output0 = outputs[0]/T1
        output1 = outputs[1]/T2
        outputs_stack = torch.stack((output0,output1),dim=0)

        outputs_mean = torch.mean(outputs_stack, dim=0)
        outputs_softmax = torch.nn.functional.softmax(outputs_mean, dim=-1)

        uncs_max, _, _, predict = calculate_predicted(outputs_softmax)

        correct = calculate_correct(predict, labels_cat)
        covarege = RC_curve(predict,labels_cat.to("cuda"),uncs_max)
        correct_list_Tdif.append(correct)
        covarege_list1_Tdif.append(covarege)

        area_sobre_curva = auc(np.arange(0.05,1.05,0.05), covarege)
        auc_list_Tdif.append(area_sobre_curva)
        #print(round((1-correct)*100,2), " ",area_sobre_curva, "Valor de Ts: ", (T1, T2))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [7]:
import pickle 

with open("./Data_sets/pickle/Cifar100/Meansoft/correct_list_Tdif.pickle", "wb") as f:
    pickle.dump(correct_list_Tdif, f)
    
with open("./Data_sets/pickle/Cifar100/Meansoft/covarege_list_Tdif.pickle", "wb") as f:
    pickle.dump(covarege_list1_Tdif, f)

with open("./Data_sets/pickle/Cifar100/Meansoft/auc_list_Tdif.pickle", "wb") as f:
    pickle.dump(auc_list_Tdif, f)